In [1]:
from recommendation_model import interface, model, preprocessing
import pandas as pd

In [2]:
from api.database_modules.entities import *
from api.models import *
from api.routers.users import get_activities

In [3]:
def query_to_value(query):
    if len(query) == 0:
        return None
    return query[0]

In [4]:
meal_survey = list(Survey.select().where(Survey.survey_type == str(SurveyType.MEAL.value)).limit(1))[0]
meal_survey_questions = list(SurveyQuestion.select().where(SurveyQuestion.survey == meal_survey.id))
meal_questions_ids = [question.question for question in meal_survey_questions]
meal_questions = list(Question.select().where(Question.id << meal_questions_ids))

print(meal_questions)

[<Question: 1>, <Question: 2>, <Question: 3>, <Question: 4>, <Question: 5>]


In [5]:
users = User.select().where(User.name == "user123")
for user in users:
    user_id = user.id
print(user_id)

user_dto = UserDto(id=user_id, name="user123", email="user123@gmail.com")

activities = get_activities(user_dto)

meals_activities = [activity for activity in activities if activity.activity_type == ActivityType.MEAL]

meals_activities_ids = [meal.activity_id for meal in meals_activities]

survey_answers = []
filtered_activities_ids = []
for activity_id in meals_activities_ids:
    activity_survey_answers = SurveyAnswer.select().where(SurveyAnswer.activity_id == activity_id)
    if len(activity_survey_answers) == 0:
        continue
    
    question_answers = [query_to_value(activity_survey_answers.where(SurveyAnswer.question == question.id).limit(1)) for question in meal_questions]

    if None in question_answers:
        continue

    survey_answers.append(question_answers)
    filtered_activities_ids.append(activity_id)

# take only the meals for which we have survey answers
user_meals = [query_to_value(UserMeal.select().where(UserMeal.id == activity_id).limit(1)) for activity_id in filtered_activities_ids]
print(user_meals)
print(survey_answers)

available_meals = list(Meal.select())
print(available_meals)

3
[<UserMeal: 12>, <UserMeal: 10>, <UserMeal: 8>]
[[<SurveyAnswer: 34>, <SurveyAnswer: 35>, <SurveyAnswer: 36>, <SurveyAnswer: 37>, <SurveyAnswer: 38>], [<SurveyAnswer: 24>, <SurveyAnswer: 25>, <SurveyAnswer: 26>, <SurveyAnswer: 27>, <SurveyAnswer: 28>], [<SurveyAnswer: 19>, <SurveyAnswer: 20>, <SurveyAnswer: 21>, <SurveyAnswer: 22>, <SurveyAnswer: 23>]]
[<Meal: 1>, <Meal: 2>, <Meal: 3>, <Meal: 4>, <Meal: 5>, <Meal: 6>, <Meal: 7>, <Meal: 8>, <Meal: 9>, <Meal: 10>, <Meal: 11>, <Meal: 12>, <Meal: 13>, <Meal: 14>, <Meal: 15>, <Meal: 16>, <Meal: 17>, <Meal: 18>, <Meal: 19>, <Meal: 20>, <Meal: 21>]


In [6]:
def prepare_user_meal(user_meal, survey_answers):
    result = {
        "id": user_meal.meal.id,
        "name": user_meal.meal.name,
        "health_index": user_meal.meal.health_index,
        "glycemic_index": user_meal.meal.glycemic_index,
        "protein": user_meal.meal.protein,
        "carbohydrates": user_meal.meal.carbohydrates,
        "fats": user_meal.meal.fats,
        "fiber": user_meal.meal.fiber,
        "meal_type": user_meal.meal.meal_type,
        "weight": user_meal.weight,
    }

    result.update(prepare_survey_answers(survey_answers))

    return result
def prepare_meal(meal):
    return {
        "id": meal.id,
        "name": meal.name,
        "health_index": meal.health_index,
        "glycemic_index": meal.glycemic_index,
        "protein": meal.protein,
        "carbohydrates": meal.carbohydrates,
        "fats": meal.fats,
        "fiber": meal.fiber,
        "meal_type": meal.meal_type,
    }
def prepare_survey_answers(survey_answers):
    return {
        f"question_{answer.question.id}": answer.answer_score
        for answer in survey_answers
    }

In [7]:
user_meals = [prepare_user_meal(user_meal, answers) for user_meal, answers in zip(user_meals, survey_answers)]
available_meals = [prepare_meal(meal) for meal in available_meals]

In [8]:
meal_target = available_meals[0]
meal_target.update({"weight": 100})

In [9]:
meal, weight, score = interface.get_meal_recommendation(user_meals, available_meals)
print(meal, weight, score)


(1, 10, 12)
1/1 [==============================] - 0s 25ms/step
{'id': 9, 'name': 'Chocolate', 'health_index': 2, 'glycemic_index': 49, 'protein': 7.65, 'carbohydrates': 59.4, 'fats': 29.7, 'fiber': 3.4, 'meal_type': 1} -0.16599973311978644 4.1745973


In [10]:
meal.name

AttributeError: 'int' object has no attribute 'name'